
# Building Multi-Agent Systems with Strands Agent Graph
Multi-agent systems leverage multiple specialized AI agents working together to solve complex problems through coordinated collaboration. Each agent has specific capabilities and roles, connected through explicit communication pathways.

In this lab, you'll learn to build multi-agent systems using the Strands Agent SDK. We'll progress from basic concepts to advanced implementations, exploring different topologies and real-world applications.

**Learning Objectives:**
By the end of this notebook, you'll be able to:
- Understand the three core components of agent graphs (nodes, edges, conditions)
- Send targeted messages between specific agents
- Monitor and control multi-agent networks
- Design specialized agent systems for real-world scenarios

## Prerequisites

- Python 3.10+
- AWS account with Anthropic Claude 3.7 enabled on Amazon Bedrock
- IAM role with permissions to use Amazon Bedrock
- Basic understanding of AI agents and prompt engineering

## Setup and Installation

Before we start, let's install the requirement packages for `strands-agents` and `strands-agents-tools`

In [1]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


### Importing required packages

Next we can import the required packages

In [2]:
from strands import Agent

## Understanding Agent Graph Components
An agent graph is a structured network of interconnected AI agents designed to solve complex problems through coordinated collaboration. Each agent represents a specialized node with specific capabilities, and the connections between agents define explicit communication pathways.

Before we start building, let's understand the three primary components of an agent graph:

### 1. Nodes (Agents)
Each node represents an AI agent with:
- **Identity**: Unique identifier within the graph
- **Role**: Specialized function or purpose
- **System Prompt**: Instructions defining the agent's behavior
- **Tools**: Capabilities available to the agent

### 2. Edges (Connections)
Edges define communication pathways with:
- **Direction**: One-way or bidirectional information flow

### Basic processing

Let's start with a simple example of one task processed by two different agents providing an output that will depend on their defined role. Take a look at the execution order of the nodes and also the fact that with STrands SDK you can explicitly get a response only from one single node if needed. Architecture looks as following:

<div style="text-align:left">
    <img src="images/basic.png" width="55%" />
</div>

In [4]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

# Create specialized agents
coordinator = Agent(name="coordinator", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0", system_prompt="You are a research team leader coordinating specialists.Provide a short analysis, no need for follow ups")
analyst = Agent(name="data_analyst", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0", system_prompt="You are a research team leader coordinating specialists.Provide a short analysis, no need for follow ups")
domain_expert = Agent(name="domain_expert", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0", system_prompt="You are a research team leader coordinating specialists.Provide a short analysis, no need for follow ups")

# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(coordinator, "team_lead")
builder.add_node(analyst, "analyst")
builder.add_node(domain_expert, "expert")

# Add edges (dependencies)
builder.add_edge("team_lead", "analyst")
builder.add_edge("team_lead", "expert")

# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("team_lead")

# Build the graph
graph = builder.build()

#Execute task on newly built graph
result = graph("Analyze the impact of remote work on employee productivity.Provide a short analysis, no need for follow ups")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")


# Get results from specific nodes
print("\n")
print("=============Expert node results only:======================")
print("============================================================")
print(result.results["expert"].result)

Graph without execution limits may run indefinitely if cycles exist


# Impact of Remote Work on Employee Productivity: Analysis

Remote work presents a nuanced productivity landscape with several key effects:

**Positive Impacts:**
- Elimination of commute time, allowing reallocation to productive work
- Reduced workplace distractions, enabling deeper focus periods
- Increased autonomy correlating with higher job satisfaction
- Schedule flexibility accommodating individual productivity patterns

**Negative Impacts:**
- Communication barriers and potential collaboration inefficiencies
- Technology challenges and connectivity issues
- Work-home boundary blurring leading to potential burnout
- Reduced social connection affecting engagement for some personality types

**Contextual Factors:**
- Job type significantly determines remote productivity outcomes
- Individual home environments create productivity disparities
- Management approaches must adapt to remote monitoring challenges
- Organizational digital infrastructure maturity dictates transition succes

### Parallel processing

Now let's create a topology when we will have 2 agents processing the request looking at 2 different aspect  of the problem and have them input into a final agent responsible for summarization and risk calculation based on provided input 
<div style="text-align:left">
    <img src="images/parallel.png" width="55%" />
</div>

In [5]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# Create specialized agents

financial_advisor = Agent(name="financial_advisor", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0", system_prompt="You are a financial advisor focused on cost-benefit analysis, budget implications, and ROI calculations. Engage with other experts to build comprehensive financial perspectives.")
technical_architect = Agent(name="technical_architect", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0",  system_prompt="You are a technical architect who evaluates feasibility, implementation challenges, and technical risks. Collaborate with other experts to ensure technical viability.")
market_researcher = Agent(name="market_researcher", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0", system_prompt="You are a market researcher who analyzes market conditions, user needs, and competitive landscape. Work with other experts to validate market opportunities.")
risk_analyst = Agent(name="risk_analyst", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0", system_prompt="You are a risk analyst who identifies potential risks, mitigation strategies, and compliance issues. Collaborate with other experts to ensure comprehensive risk assessment.")


# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(financial_advisor, "finance_expert")
builder.add_node(technical_architect, "tech_expert")
builder.add_node(market_researcher, "market_expert")
builder.add_node(risk_analyst, "risk_analyst")

# Add edges (dependencies)
builder.add_edge("finance_expert", "tech_expert")
builder.add_edge("finance_expert", "market_expert")
builder.add_edge("tech_expert", "risk_analyst")
builder.add_edge("market_expert", "risk_analyst")


# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("finance_expert")

# Build the graph
graph = builder.build()

print("============================================================")
print("============================================================")

#Execute task on newly built graph
result = graph("Our company is considering launching a new AI-powered customer service platform. Initial investment is \$2M with projected 3-year ROI of 150%. What's your financial assessment?")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")


# Get results from specific nodes

print("Financial Advisor:")
print("============================================================")
print("============================================================")
print(result.results["finance_expert"].result)
print("\n")

print("Technical Expert:")
print("============================================================")
print("============================================================")
print(result.results["tech_expert"].result)
print("\n")

print("Market Researcher:")
print("============================================================")
print("============================================================")
print(result.results["market_expert"].result)
print("\n")

<>:39: SyntaxWarning: invalid escape sequence '\$'
<>:39: SyntaxWarning: invalid escape sequence '\$'
C:\Users\mnede\AppData\Local\Temp\ipykernel_23944\2348947990.py:39: SyntaxWarning: invalid escape sequence '\$'
  result = graph("Our company is considering launching a new AI-powered customer service platform. Initial investment is \$2M with projected 3-year ROI of 150%. What's your financial assessment?")
Graph without execution limits may run indefinitely if cycles exist


# Financial Assessment of AI Customer Service Platform

This investment presents promising returns, but I'd like to break down some key financial considerations:

## Initial Analysis
- $2M upfront investment
- 150% projected ROI over 3 years
- This implies a return of approximately $5M over 3 years ($2M investment + $3M profit)

## Questions to Consider
- What's the expected annual revenue/cost savings breakdown across the 3 years?
- Have you accounted for ongoing maintenance costs in the ROI calculation?
- What's the projected payback period within those 3 years?

## Risk Considerations
- The ROI projection appears strong, but I'd want to validate the assumptions behind these figures
- Technology obsolescence risk could affect long-term value
- Implementation delays could postpone return realization

A 150% 3-year ROI generally indicates a sound investment, but I'd recommend creating a detailed cash flow analysis with quarterly projections to better understand the investment trajector

### Branching with conditions

Let's create an agent graph that would classify the request and depending on conditions we define in the code - will route the request either to technical or business agent.

Take a close look on differences between the node execution order and number of nodes executed in this graph based on two different prompts.

<div style="text-align:left">
    <img src="images/conditional.png" width="55%" />
</div>

In [ ]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# Create specialized agents

classifier = Agent(name="classifier", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0", system_prompt="You are an agent responsible for classification of the report request, return only Technical or Business classification.")
technical_report = Agent(name="technical_expert", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0", system_prompt="You are a technical expert htat focuses on providing short summary from technical perspective")
business_report = Agent(name="business_expert", model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0", system_prompt="You are a business expert that focuses on providing short summary from business perspective")

# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(classifier, "classifier")
builder.add_node(technical_report, "technical_report")
builder.add_node(business_report, "business_report")

def is_technical(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "technical" in result_text.lower()

def is_business(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "business" in result_text.lower()

# Add edges (dependencies)
builder.add_edge("classifier", "technical_report", condition=is_technical)
builder.add_edge("classifier", "business_report", condition=is_business)

# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("classifier")

# Build the graph
graph = builder.build()

print("============================================================")
print("============================================================")

#Execute task on newly built graph
result = graph("Provide report on technical aspect of working from home, outline things to consider and key risk factors")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

# Get results from specific nodes

print("Classifier:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

#Execute task on newly built graph
result = graph("Provide report on business impact of working from home, outline things to consider and key risk factors")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

# Get results from specific nodes

print("Classifier:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

Graph without execution limits may run indefinitely if cycles exist


Technical# Technical Report: Working from Home Considerations and Risk Factors

## Technical Infrastructure Requirements

1. **Network Connectivity**
   - Reliable high-speed internet connection (minimum 25/5 Mbps recommended)
   - Backup connectivity solution (mobile hotspot, secondary ISP)
   - VPN capabilities for secure remote access

2. **Hardware Setup**
   - Adequate computing devices (laptop/desktop with sufficient processing power)
   - Peripheral equipment (monitors, keyboard, mouse, webcam, headset)
   - Ergonomic considerations for prolonged use

3. **Software Requirements**
   - Remote access tools and licenses
   - Collaboration platforms (video conferencing, messaging)
   - Security software (antivirus, anti-malware)

## Key Risk Factors

1. **Cybersecurity Vulnerabilities**
   - Unsecured home networks exposing company data
   - Personal device usage lacking proper security controls
   - Increased phishing attack surface

2. **Technical Support Challenges**
   - Limited

## Key Takeaways and Best Practices


### Best Practices:

**Design for acyclicity:** Ensure your graph has no cycles</p>
**Use meaningful node IDs:** Choose descriptive names for nodes</p>
**Validate graph structure:** The builder will check for cycles and validate entry points</p>
**Handle node failures:** Consider how failures in one node affect the overall workflow</p>
**Use conditional edges:** For dynamic workflows based on intermediate results</p>
**Consider parallelism:** Independent branches can execute concurrently</p>
**Nest multi-agent patterns:** Use Swarms within Graphs for complex workflows</p>
**Leverage multi-modal inputs:** Use ContentBlocks for rich inputs including images</p>

## Conclusion

You've now mastered the fundamentals of building multi-agent systems with Strands Agent Graph! You can create sophisticated networks of specialized AI agents that collaborate to solve complex problems.

The key to successful multi-agent systems is:
- Matching topology to use case
- Defining clear agent roles and responsibilities  
- Establishing proper communication patterns
- Managing resources and cleanup effectively

From here, you can build increasingly sophisticated systems for real-world applications in research, content creation, decision-making, customer service, and beyond.